In [48]:
import cv2
import numpy as np

In [49]:
# Function to organize matched keypoints into two sets (Arrange Pairs Function)
def organize_matched_points(matches, keypoints1, keypoints2):
    points1 = []
    points2 = []
    for match in matches:
        points1.append(keypoints1[match.queryIdx].pt)
        points2.append(keypoints2[match.trainIdx].pt)
    return (points1, points2)

In [50]:
# Function to apply a transformation matrix to a set of points (apply transformation function)
def transform_points(transformation_matrix, points_set):
    transformed_points = []
    for x, y in points_set:
        x_transformed = (
            transformation_matrix[0][0] * x
            + transformation_matrix[0][1] * y
            + transformation_matrix[0][2]
        )
        y_transformed = (
            transformation_matrix[1][0] * x
            + transformation_matrix[1][1] * y
            + transformation_matrix[1][2]
        )
        transformed_points.append([x_transformed, y_transformed])
    return transformed_points

In [51]:
# Function to calculate the number of inliers for a given transformation
def count_inliers(transformation_matrix, points1, points2, threshold=10):
    inliers_count = 0
    for i in range(len(points1)):
        x, y = points1[i]
        x_transformed = (
            transformation_matrix[0][0] * x
            + transformation_matrix[0][1] * y
            + transformation_matrix[0][2]
        )
        y_transformed = (
            transformation_matrix[1][0] * x
            + transformation_matrix[1][1] * y
            + transformation_matrix[1][2]
        )
        x_target, y_target = points2[i]
        distance = np.sqrt(
            (x_transformed - x_target) ** 2 + (y_transformed - y_target) ** 2
        )
        if distance < threshold:
            inliers_count += 1
    return inliers_count

In [52]:
# Function to warp and blend two images using a homography matrix
def blend_images(image1, image2, homography_matrix):
    height1, width1 = image1.shape[:2]
    height2, width2 = image2.shape[:2]
    corners1 = np.float32(
        [[0, 0], [0, height1], [width1, height1], [width1, 0]]
    ).reshape(-1, 1, 2)
    corners2 = np.float32(
        [[0, 0], [0, height2], [width2, height2], [width2, 0]]
    ).reshape(-1, 1, 2)
    transformed_corners2 = cv2.perspectiveTransform(corners2, homography_matrix)
    all_corners = np.concatenate((corners1, transformed_corners2), axis=0)
    [x_min, y_min] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(all_corners.max(axis=0).ravel() + 0.5)
    translation_vector = [-x_min, -y_min]
    translation_matrix = np.array(
        [[1, 0, translation_vector[0]], [0, 1, translation_vector[1]], [0, 0, 1]]
    )

    result_image = cv2.warpPerspective(
        image2,
        translation_matrix.dot(homography_matrix),
        (x_max - x_min, y_max - y_min),
    )
    result_image[
        translation_vector[1] : height1 + translation_vector[1],
        translation_vector[0] : width1 + translation_vector[0],
    ] = image1
    return result_image

In [53]:
# Function to estimate the transformation matrix between two sets of points (custom RANSAC)
def estimate_transformation(points1, points2):
    best_inliers_count = 0
    best_transformation = []

    for _ in range(50):
        random_indices = np.random.choice(len(points1), 10)
        subset_points1 = [points1[i] for i in random_indices]
        subset_points2 = [points2[i] for i in random_indices]

        matrix = []
        for x, y in subset_points1:
            row1 = [x, y, 0, 0, 1, 0]
            row2 = [0, 0, x, y, 0, 1]
            matrix.extend([row1, row2])

        matrix = np.tile(matrix, (1, 1))

        A = matrix.copy()
        B = []
        for x, y in subset_points2:
            B.append(x)
            B.append(y)

        A_transpose = np.transpose(A)
        A_transpose_A = np.dot(A_transpose, A)
        A_transpose_B = np.dot(A_transpose, B)
        coefficients = np.dot(np.linalg.inv(A_transpose_A), A_transpose_B)
        transformation_matrix = np.array(
            [
                [coefficients[0], coefficients[1], coefficients[4]],
                [coefficients[2], coefficients[3], coefficients[5]],
            ]
        )
        inliers_count = count_inliers(transformation_matrix, points1, points2)
        if inliers_count > best_inliers_count:
            best_inliers_count = inliers_count
            best_transformation = transformation_matrix

    best_transformation = np.vstack([best_transformation, [0, 0, 1]])
    return best_transformation

In [54]:
# Load and preprocess images
image1 = cv2.imread("./ImageA.jpg")
image2 = cv2.imread("./ImageB.jpg")
gray_image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
gray_image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

In [55]:
# Extract features and match them
feature_extractor = cv2.xfeatures2d.SIFT_create()
(keypoints1, descriptors1) = feature_extractor.detectAndCompute(gray_image1, None)
(keypoints2, descriptors2) = feature_extractor.detectAndCompute(gray_image2, None)

matcher = cv2.BFMatcher()
matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

good_matches = []
for m, n in matches:
    if m.distance / n.distance < 0.8:
        good_matches.append(m)

In [56]:
# Organize matched points and visualize matches
points1, points2 = organize_matched_points(good_matches, keypoints1, keypoints2)
matched_image = cv2.drawMatches(
    image1,
    keypoints1,
    image2,
    keypoints2,
    good_matches,
    None,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
)
cv2.imshow("Matched Features", matched_image)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread

In [57]:
# Estimate homography and transformation matrices
homography_matrix, _ = cv2.findHomography(
    np.array(points2), np.array(points1), cv2.RANSAC, 5.0
)
blended_image = blend_images(image1, image2, homography_matrix)
cv2.imshow("Blended Image", blended_image)

transformation_matrix = estimate_transformation(points2, points1)

# Warp and blend images using the estimated transformation
image1_transformed = image1.copy()
cv2.warpPerspective(
    image1,
    transformation_matrix,
    (image1.shape[1], image1.shape[0]),
    image1_transformed,
    cv2.INTER_LINEAR,
    cv2.BORDER_CONSTANT,
    0,
)

blended_image = blend_images(image1, image2, transformation_matrix)
cv2.imshow("Blended Image", blended_image)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread (0x326b950).
Cannot move to target thread (0x1db7080)

QObject::moveToThread: Current thread (0x1db7080) is not the object's thread

In [58]:
# Wait for a key press and close all OpenCV windows
cv2.waitKey(0)
cv2.destroyAllWindows()